# Results plots
Script for plotting all the results of the integration to check what happened
## Table of Contents <a class="anchor" id="top"></a>
* [Temperature and flux](#temp)
* [Core central temperature](#Tc)
* [Inner core size and Rem](#ic)
* [Rayleigh number](#rayleigh)
* [Stagnant lid thickness](#stagnant)
* [CMB mantle boundary layer thickness](#cmb-bl)
* [Sulfur fraction in liquid core](#sulfur)
* [Initial temp profiles](#Tprofile-initial)
* [Temperature profiles with time](#Tprofile-time)

## Set-up

In [1]:
#import modules
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from load_info import load_run_info 

#scale time to Myr
from parameters import Myr, r, dr, rc

ImportError: cannot import name 'Tm0' from 'parameters' (/home/users/exet5460/Documents/Code/learning-model/Version4/parameters.py)

Choose your run and whether you want to save your figures

In [ ]:
run=1
conduction = False # does the mantle switch to conduction?
save = False # do you want to save your figures?

Import data from npz file

In [ ]:
npzfile = np.load('Results_combined/run_{}.npz'.format(run))
Tc= npzfile['Tc'] 
Tc_conv = npzfile['Tc_conv']
Tcmb = npzfile['Tcmb']
Tm_mid = npzfile['Tm_mid']
Tm_conv = npzfile['Tm_conv']
Tm_surf = npzfile['Tm_surf'] 
T_profile = npzfile['T_profile']
f = npzfile['f'] 

t = npzfile['t'] #time in s
Rem1 = npzfile['Rem1'] # magnetic Reynolds number from compositional (Nimmo) and thermal convection (whatever is larger at each time step)
Rem2 = npzfile['Rem2'] # magnetic Reynolds number from compositional (Nichols) and thermal convection (whatever is larger at each time step)
Flux = npzfile['Flux']
Ra = npzfile['Ra'] 
RaH = npzfile['RaH'] 
RanoH = npzfile['RanoH'] 
Racrit = npzfile['Racrit'] 
d0 = npzfile['d0'] 
min_unstable = npzfile['min_unstable'] 
Xs = npzfile['Xs']
dl = npzfile['dl']
dx = npzfile['dc']
Fs = Flux[0]
Fcmb = Flux[1]
Fad = Flux[2]
Frad = Flux[3]

Scale time to Myr

In [ ]:
t_plot = t/Myr
log_time = True # do you want to plot time logarithmically

Import label info - read in from correct row in csv. If using run_info3.csv need to remember there is an extra Tsolidus column

In [ ]:
r, dr, tstart, tend, tstep, tsolid, cond_t, dt, viscosity = load_run_info(2,'run_info.csv',conduction)

# Temperature profile

In [ ]:
rplot= np.arange(0,r,dr)
r_unstable=np.array([]) 
for ind in min_unstable:
    r_unstable = np.append(r_unstable,rplot[int(ind)])

In [ ]:
plt.figure()
plt.pcolormesh(t/Myr, rplot/1e3,np.transpose(T_profile),shading = 'nearest') # use same vmin and vmax as other plots to make them comparable
plt.hlines(rc/1e3,min(t/Myr),max(t/Myr),linestyle='--',color='black',label='CMB')
plt.plot(t/Myr,r_unstable/1e3,linestyle='dotted',label='Convecting core')
if conduction == True:
    plt.plot(t[:cond_i]/Myr,(r-d0[:cond_i])/1e3,linestyle='dashed',label='base of $\delta_0$',color='blue')
else:
    plt.plot(t/Myr,(r-d0)/1e3,linestyle='dashed',label='base of $\delta_0$',color='blue')
plt.xscale('log')
plt.ylabel('r /km')
plt.xlabel('t / Myr')
plt.ylim([0,r/1e3])
#plt.xlim([0.5,0.7])
plt.colorbar(label='T/K')
plt.title(f'Run {run}')
plt.legend(loc='lower left')
if save == True:
    plt.savefig(f'Plots/Tdiff_run{run}.png')
#plt.savefig(f'Plots/Dodds_fig5_run{run}.png')

## Temperature and Flux Plot <a class="anchor" id="temp"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure(tight_layout=True,figsize=[10,7])
plt.suptitle(f'Thermal evolution of a {r/1e3:.0f}km asteroid \n Tm0 = {Tm0}K,\n run {run} viscosity {viscosity}')

xmin=tstart

#temperatures as function of time
plt.subplot(2,1,1)
if log_time == True:
    plt.semilogx(t_plot,Tc,label='central T$_c$')
    plt.semilogx(t_plot[Tc_conv!=0],Tc_conv[Tc_conv!=0],label='convective T$_c$')
    plt.semilogx(t_plot,Tcmb,label='T$_{cmb}$')
    plt.semilogx(t_plot,Tm_mid,label='T$_m$')
    plt.semilogx(t_plot[Tm_conv!=0],Tm_conv[Tm_conv!=0],label='convective T$_m$')
    #plt.semilogx(t_plot,Tm_surf,label='T$_m$ - surface')
else:
    plt.plot(t_plot,Tc,label='central T$_c$')
    plt.plot(t_plot[Tc_conv!=0],Tc_conv[Tc_conv!=0],label='convective T$_c$')
    plt.plot(t_plot,Tcmb,label='T$_{cmb}$')
    plt.plot(t_plot,Tm_mid,label='T$_m$')
    plt.plot(t_plot[Tm_conv!=0],Tm_conv[Tm_conv!=0],label='convective T$_m$')
    #plt.plot(t_plot,Tm_surf,label='T$_m$ - surface')
if conduction == True:
    plt.vlines(cond_t,ymin=min(Tm_surf),ymax=1600,color='black',linestyle='--',label='conduction')
plt.xlim(right=50)
plt.ylim([1460,1650]) #use these limits when comparing runs
plt.ylabel('T/K')
plt.legend(loc='lower left', ncol= 2)

#fluxes as function of time
plt.subplot(2,1,2)

Fcmb_neg = Fcmb[Fcmb<0]
Fcmb_pos = Fcmb[Fcmb>0]
if log_time == True:
    plt.loglog(t_plot,Fs,label='$F_s$')
    plt.loglog(t_plot[Fcmb<0],abs(Fcmb_neg),label='$-F_{CMB}$',linestyle='dashed',color='purple')
    plt.loglog(t_plot[Fcmb>0],Fcmb_pos,label='$F_{CMB}$',color='purple')
    plt.loglog(t_plot,Fad,label='$F_{ad}$')
    plt.loglog(t_plot,Frad,label='$F_{rad}$')
else:
    plt.semilogy(t_plot,Fs,label='$F_s$')
    plt.semilogy(t_plot[Fcmb<0],abs(Fcmb_neg),label='$-F_{CMB}$',linestyle='dashed',color='purple')
    plt.semilogy(t_plot[Fcmb>0],Fcmb_pos,label='$F_{CMB}$',color='purple')
    plt.semilogy(t_plot,Fad,label='$F_{ad}$')
    plt.semilogy(t_plot,Frad,label='$F_{rad}$')
plt.xlabel('Time/ Myr')

plt.ylim([1e-4,1e2])   #use these limits when comparing runs
plt.xlim(right=50)
plt.ylabel('Flux/ W$m^{-2}$')
plt.legend(loc='upper right',ncol=2)


if save == True:
    plt.savefig('Plots/run_{}_Tflux.png'.format(run),dpi=450)


### Comments
There is a jump in $F_S$ and $F_{CMB}$ when mantle convection stops. I think $F_S$ increases because the length scale of the temperature drop is smaller (goes from $\delta_0$ to dr)? Not sure why there is a weird spike in Tm_surf - seems like it jumps to interior temp for a moment, but there is no change in $\delta_0$ at that point

## Core central temperature <a class="anchor" id="Tc"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure()
plt.plot(t_plot,Tc)
#plt.xlim([5.9,6])
plt.xlabel('Time/Myr')
plt.ylabel('T/K')
#plt.savefig('Plots/Tc_fe8e7.png')

## Magnetic Reynolds number and inner core size <a class="anchor" id="ic"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure(tight_layout=True)
plt.suptitle(f'Thermal evolution of a {r/1e3:.0f}km asteroid \n Tm0 = {Tm0}K, \n run {run}  {viscosity}')
plt.subplot(2,1,1)
plt.loglog(t_plot,Rem1,label='Nimmo')
plt.loglog(t_plot,Rem2,label='Nichols')
plt.hlines(10,xmin=0,xmax=t_plot[len(Rem1)-1],color='k',linestyle='--')
#plt.xlabel('Time/Myr')
plt.ylabel('Rem')
plt.legend(loc='upper left')
plt.ylim([1,100])

plt.subplot(2,1,2)
plt.semilogx(t_plot,f,label='f')
plt.xlabel('Time/ Myr')
plt.ylabel('f')

if save == True:
    plt.savefig('Plots/run_{}_Remf.png'.format(run),dpi=450)

## Rayleigh number<a class="anchor" id="rayleigh"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure()
if conduction == True:
    plt.semilogy(t_plot[:cond_i],Ra[:cond_i])
    plt.semilogy(t_plot[:cond_i],Racrit[:cond_i],color='k',linestyle='--',label='Critical Rayleigh number')
else:
    plt.semilogy(t_plot,Ra)
    plt.semilogy(t_plot,Racrit,color='k',linestyle='--',label='Critical Rayleigh number')
plt.xlabel('Time/Myr')
plt.ylabel('Rayleigh number')
plt.legend()
#plt.ylim([6e6,1e7])

if save == True:
    plt.savefig('Plots/run_{}_Rayleigh.png'.format(run))

Compare the Rayleigh number output by the simulation with the non-radiogenic Ra

In [ ]:
#save = True

In [ ]:
plt.figure()
if conduction == True:
    
    plt.semilogy(t_plot[:cond_i],RaH[:cond_i],label='Radiogenic Ra')
    plt.semilogy(t_plot[:cond_i],RanoH[:cond_i],color='green',label='Non Radiogenic Ra')
    plt.semilogy(t_plot[:cond_i],Racrit[:cond_i],color='k',linestyle='--',label='Critical Rayleigh number')
    plt.semilogy(t_plot[:cond_i],Ra[:cond_i],linestyle='--',color='blue',label='Model Ra')
else:
    plt.semilogy(t_plot,RaH,label='Radiogenic Ra')
    plt.semilogy(t_plot,RanoH,color='green',label='Non Radiogenic Ra')
    plt.semilogy(t_plot,Racrit,color='k',linestyle='--',label='Critical Rayleigh number')
    plt.semilogy(t_plot,Ra,linestyle='--',color='blue',label='Model Ra')
plt.xlabel('Time/Myr')
plt.ylabel('Rayleigh number')
plt.legend()
#plt.ylim([1e4,1e25])
#plt.xlim([0,40])
if save == True:
    plt.savefig('Plots/run_{}_Rayleigh.png'.format(run))

### Comments
Convection stops much earlier now $Ra_c$ is higher (9.4Myr compared to 30Myr).

## Stagnant lid thickness <a class="anchor" id="stagnant"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure()

if conduction == True:
    plt.plot(t_plot[:cond_i],2*d0[:cond_i]/r)
    plt.hlines(1,min(t_plot),max(t_plot[:cond_i]),color='k',linestyle='--')
    #plt.xlim([xmin,t_plot[cond_i]])
    
else:
    plt.plot(t_plot,2*d0/r)
    plt.hlines(1,min(t_plot),max(t_plot),color='k',linestyle='--')

plt.xlabel('Time/Myr')
plt.ylabel('Stagnant Lid thickness/ mantle thickness')

if save == True:
    plt.savefig('Plots/run_{}_lid_thickness.png'.format(run))

## CMB mantle boundary layer thickness<a class="anchor" id="cmb-bl"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure()

if conduction == True:
    plt.plot(t_plot[:cond_i],2*dl[:cond_i]/dr)
    #plt.hlines(1,min(t_plot),max(t_plot[:cond_i]),color='k',linestyle='--')
    plt.xlim([xmin,t_plot[cond_i]])
else:
    plt.semilogy(t_plot,2*dl/dr)
plt.xlabel('Time/Myr')
plt.ylabel('CMB boundary layer thickness/ cell thickness')
if save == True:
    plt.savefig('Plots/run_{}_cmb_thickness.png'.format(run))


## Sulfur fraction in liquid part of core <a class="anchor" id="sulfur"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
plt.figure()
#plt.loglog(t_plot,Xs)
plt.scatter(t_plot,Xs)
#plt.xlim([297.5,299.5])
#plt.xlim([xmin,max(t_plot)])
plt.xlabel('Time/Myr')
plt.ylabel('Liquid core sulfur content/ wt%')

if save == True:
    plt.savefig('Plots/run_{}_sulfur.png'.format(run))

## Initial temperature profiles <a class="anchor" id="Tprofile-initial"></a>
<p align="right">(<a href="#top">back to top</a>)</p>
I don't think the legend here is quite right.

In [ ]:
plt.figure()
for i in range(1):
    plt.plot(rplot/1e3, T_profile[i,:],label=f'{(1+i*dt)/Myr:.1e} Myr') #approximate temp profile times
plt.xlabel('Distance from centre of asteroid /km')
plt.ylabel('Temperature / K')
plt.title('{:.0f}km asteroid \n Tm0 = {}K, run {}'.format(r/1e3, Tm0, run))
plt.legend()
if save == True:
    plt.savefig('Plots/run_{}_initial_temp.png'.format(run))

## Temperature profiles across whole simulation <a class="anchor" id="Tprofile-time"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

In [ ]:
from parameters import out_interval
out_interval = out_interval/Myr

In [ ]:
plt.figure()
rplot= np.arange(0,r,dr)
n = np.shape(T_profile)[0]
n_plot = 10 #how many plots do you want
for i in range(n_plot):
#for i in range(15):
    plt.scatter(rplot/1e3, T_profile[i*int(n/n_plot),:],label='{:.0f} Myr'.format(i*int(n/n_plot)*out_interval)) #approximate temp profile times
plt.xlabel('Distance from centre of asteroid /km')
plt.ylabel('Temperature / K')
plt.title(f'{r/1e3:.0f}km asteroid \n Tm0 = {Tm0}K, run {run} {viscosity}')
plt.legend(ncol=2,loc='lower left')
#plt.vlines(2,ymin=1590,ymax=1625,linestyle='--',color='black')
#plt.xlim([0,210])
#plt.xlim([0,25])
#plt.ylim([1595,1615])
if save == True:
    plt.savefig('Plots/run{}_Tprofile.png'.format(run))